In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, coint
import statsmodels.api as sm

import os
from datetime import datetime

<h2 style="color:purple;">Kalman Filters</h2>

In [2]:
from pykalman import KalmanFilter
from math import sqrt

In [5]:
# faster Kalman Filter from IterativeBacktester can be implemented
def KalmanFilterAverage(x):
  # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],
    observation_matrices = [1],
    initial_state_mean = 0,
    initial_state_covariance = 1,
    observation_covariance=1,
    transition_covariance=.01)
  # Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(x.values)
    state_means = pd.Series(state_means.flatten(), index=x.index)
    return state_means
# Kalman filter regression
def KalmanFilterRegression(x,y):
    delta = 1e-3
    trans_cov = delta / (1 - delta) * np.eye(2) # How much random walk wiggles
    obs_mat = np.expand_dims(np.vstack([[x], [np.ones(len(x))]]).T, axis=1)
    kf = KalmanFilter(n_dim_obs=1, n_dim_state=2, # y is 1-dimensional, (alpha, beta) is 2-dimensional
    initial_state_mean=[0,0],
    initial_state_covariance=np.ones((2, 2)),
    transition_matrices=np.eye(2),
    observation_matrices=obs_mat,
    observation_covariance=2,
    transition_covariance=trans_cov)
    # Use the observations y to get running estimates and errors for the state parameters
    state_means, state_covs = kf.filter(y.values)
    return state_means
def half_life(spread):
    spread_lag = spread.shift(1)
    spread_lag.iloc[0] = spread_lag.iloc[1]
    spread_ret = spread - spread_lag
    spread_ret.iloc[0] = spread_ret.iloc[1]
    spread_lag2 = sm.add_constant(spread_lag)
    model = sm.OLS(spread_ret,spread_lag2)
    res = model.fit()
    halflife = int(round(-np.log(2) / res.params[1],0))
    if halflife <= 1:
        halflife = 1
    return halflife

<h2 style="color:lime;">Coint_Analyzer</h2>

In [9]:
class Coint_Analyzer:
  # raw_data_path, save_dir_path
  def __init__(self, dir_paths=["Binance_Historical_15m_FUTURES_20_days_2022-07-14T12:00:43"], observations_low_pass = 0):
    self.dir_paths = dir_paths
    self.observations_low_pass = observations_low_pass
    
    self.df = None
    self.corr_pairs = None
    self.coint_pairs = None
    self.corr_coint_pairs = None
    
    self._closings_csv_to_df()
    
  def process_raw_data(self, dir_paths=None):
    if dir_paths is not None:
      self.dir_paths = dir_paths
    
    self._raw_to_processed()
    self.get_trading_pairs()
  
  def _closings_csv_to_df(self):
    # reading Close values and merging to one DF
    df_closings = pd.DataFrame()
    
    for path in self.dir_paths:
      with os.scandir('../raw_data/%s' % path) as entries:
          for entry in entries:
            instrument = "_".join(entry.name.split("_")[0:2])
            df = pd.read_csv('../raw_data/%s/%s' % (path, entry.name), index_col="Date")
            df = df[["Close"]].copy()
            df.columns = [instrument]
            df_closings = pd.concat([df_closings, df], axis=1)
    
    # filtering data based on amount of observations in DF
    df_observation_num = pd.DataFrame(columns=["observations"])
    for column in df_closings.columns:
      df_observation_num.loc[column] = len(df_closings[column].dropna())

    drop_columns = []
    for _, row in df_observation_num.iterrows():
      if row.observations < self.observations_low_pass: # arbitrarily selected value based on bottom values from df_observation_num
        drop_columns.append(row.name)
        
    # removing outliers from the original DF
    df_closings.drop(columns=drop_columns, inplace=True)

    # cleaning DF
    df_closings.dropna(inplace=True)
            
    self.df = df_closings
    
  def _raw_to_processed(self):
    # CORRELATION
    if self.df is None:
      return
    
    matrix = self.df.pct_change().corr(method ='pearson')
    matrix.to_excel("processed_data/corr_matrix_temp_%s.xlsx" % str(datetime.utcnow().replace(microsecond=0).isoformat()))
    
    au_corr = matrix.corr().unstack()
    labels_to_drop = self._get_redundant_corr_pairs(matrix)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    au_corr.dropna(inplace=True)
    
    indexes = []
    values = []
    for idx in au_corr.index:
        indexes.append("%s-%s" % (idx[0], idx[1]))
        values.append(au_corr[idx])
    corr_pairs_df = pd.DataFrame(index=indexes, data=values)
    
    self.corr_pairs = corr_pairs_df
    try:
        corr_pairs_df.to_csv("../processed_data/corr_pairs__tem%s.csv" % str(datetime.utcnow().replace(microsecond=0).isoformat()))
    except:
      print("Couldn't save pairs to temp files")
      
    # COINTEGRATION 
    _, coint_pairs = self._find_cointegrated_pairs(self.df.copy())
    self.coint_pairs = coint_pairs
    
    
  def _get_redundant_corr_pairs(self, df_corr_matrix):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df_corr_matrix.columns
    for i in range(0, df_corr_matrix.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop
    
  def _find_cointegrated_pairs(self, df):
    n = df.shape[1]
    pvalue_matrix = np.ones((n, n))
    keys = df.copy().keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            print("Performing coint test %s %s %s" % (j, i, n))
            
            result = coint(df[keys[i]], df[keys[j]])
            pvalue_matrix[i, j] = result[1]
            
            # testing for spread stationarity
            if result[1] < 0.05:
              state_means = KalmanFilterRegression(KalmanFilterAverage(df[keys[i]]), KalmanFilterAverage(df[keys[j]]))
              hedge_ratio = - state_means[:,0]
              spread = df[keys[j]] + (df[keys[i]] * hedge_ratio)
              result_adf = adfuller(spread)
              
              if result_adf[1] < 0.01 and result_adf[0] < result_adf[4]["1%"]:
                # Granger causality test
                # maxlag value should be investigated
                # why halflife f() didn't want to work :thinking_face:
                try:
                  g12_pval = grangercausalitytests(df[[keys[i], keys[j]]], maxlag=1, verbose=False)[1][0]['ssr_chi2test'][1]
                  g21_pval = grangercausalitytests(df[[keys[j], keys[i]]], maxlag=1, verbose=False)[1][0]['ssr_chi2test'][1]
                except:
                  g12_pval = 0
                  g21_pval = 0
                  
                # is it mean reverting
                hurst = self._get_hurst_exponent(np.array(spread))
                if hurst <= 0.5:
                  pairs.append((keys[i], keys[j], result[1], result_adf[0], hurst, g12_pval, g21_pval))
    try:
        indexes = []
        adf = []
        hurst = []
        granger_12 = []
        granger_21 = []
        for row in pairs:
            indexes.append("%s-%s" % (row[0], row[1]))
            adf.append(row[3])
            hurst.append(row[4])
            granger_12.append(row[5])
            granger_21.append(row[6])
            
        coint_pairs_df = pd.DataFrame(index=indexes)
        coint_pairs_df['adf'] = adf
        coint_pairs_df['hurst'] = hurst
        coint_pairs_df['granger_12'] = granger_12
        coint_pairs_df['granger_21'] = granger_21
        coint_pairs_df.sort_values(ascending=True, by="adf")
        coint_pairs_df.to_csv("../processed_data/coint_pairs_temp%s.csv" % str(datetime.utcnow().replace(microsecond=0).isoformat()))
        
        pv_val_df = pd.DataFrame(pvalue_matrix)
        pv_val_df.columns = df.columns
        pv_val_df.index = df.columns
        pv_val_df.to_excel("../processed_data/coint_matrix_temp_%s.xlsx" % str(datetime.utcnow().replace(microsecond=0).isoformat()))
    except:
       print("Couldn't save pairs to temp files") 
    return pvalue_matrix, coint_pairs_df
  
  def _get_hurst_exponent(self, time_series):
    """Returns the Hurst Exponent of the time series vector ts"""
    # Create the range of lag values
    lags = range(2, 20)
    # Calculate the array of the variances of the lagged differences
    tau = [np.sqrt(np.std(np.subtract(time_series[lag:], time_series[:-lag]))) for lag in lags]
    # Use a linear fit to estimate the Hurst Exponent
    poly = np.polyfit(np.log(lags), np.log(tau), 1)
    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0
    
  def get_trading_pairs(self, h_pass = 0.95, corr_path = None, coint_path = None):
    df_corr = None
    df_coint = None
    if corr_path is not None and coint_path is not None:
      df_corr = pd.read_csv(corr_path)
      df_coint = pd.read_csv(coint_path)
    elif self.corr_pairs is not None and self.coint_pairs is not None:
      df_corr = self.corr_pairs.copy()
      df_coint = self.coint_pairs.copy()
      
    if df_corr is None or df_coint is None:
      return
    
    df_hi_corr = df_corr.loc[df_corr[0]>h_pass]
    df_corr_coint_pairs = pd.DataFrame(columns=["corr", "adf", "hurst", "granger_12", "granger_21"])
    for idx in df_hi_corr.index:
      if idx in df_coint.index:
        df_corr_coint_pairs.loc[idx] = [df_hi_corr.loc[idx][0], df_coint.loc[idx][0], df_coint.loc[idx][1], df_coint.loc[idx][2], df_coint.loc[idx][3]]
        
    self.corr_coint_pairs = df_corr_coint_pairs
    try:
      df_corr_coint_pairs.to_csv("../processed_data/corr_coint_pairs_temp_%s.csv" % str(datetime.utcnow().replace(microsecond=0).isoformat()))
    except:
      print("Data couldn't be stored in a static file.")
    return df_corr_coint_pairs

<h2 style="color:lime;">DEMO</h2>

In [10]:
analyzer = Coint_Analyzer(observations_low_pass=300)

In [ ]:
analyzer.process_raw_data()

In [ ]:
analyzer.get_trading_pairs()

In [ ]:
analyzer.corr_coint_pairs.loc[(analyzer.corr_coint_pairs.granger_12 <= 0.05) | (analyzer.corr_coint_pairs.granger_21 <= 0.05)]